In [1]:
# leitura dos dados
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

from pandas.api.types import CategoricalDtype
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# funções + embeddings
from src.categorizador_batch import categorizar_coluna_batch, preparar_para_categoria

# mapeamentos
from src.categorizar_texto import (
    mapeamento_motivo_fluencia,
    mapeamento_profissoes,
    mapeamento_outros_idiomas,
    mapeamento_problema_aprender,
    mapeamento_escolaridade,
    expandir_mapeamento
)

df_pesquisa_captacao_l3_24 = pd.read_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\dados\[L3-24] Pesquisa captação.csv")
df_pesquisa_captacao_l4_24 = pd.read_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\dados\[L4-24] Pesquisa captação.csv")
df_pesquisa_captacao_l1_25 = pd.read_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\dados\[L1-25] Pesquisa CAPTAÇÃO.csv")

In [2]:
df_pesquisa_captacao_l3_24 = df_pesquisa_captacao_l3_24.drop([
    "Seu whatsapp com DDD (opcional)", 
    "Você já está no grupo do whatsapp do evento? (caso não, confira seu e-mail)",
    "Qual o maior problema que você enfrenta HOJE por ainda não ser fluente em Espanhol? (exemplo: perder uma promoção no trabalho, não conseguir viajar para fora sem depender de alguém). ",
    "Conte para gente: como você já tentou resolver esse problema, e porque não teve sucesso?",
    'Caso tenha respondido "sim", descreva um pouco sobre a sua experiência, dificuldades, pontos positivos e negativos, e o quanto gastou com esse curso (sem citar nomes)',
    "Qual é a sua maior motivação para querer se tornar fluente em espanhol?",
    'Você já participou de alguma semana de aulas gratuitas do VHE?'
], axis=1, errors='ignore')

df_pesquisa_captacao_l4_24 = df_pesquisa_captacao_l4_24.drop([
    "Seu whatsapp com DDD",
    "Conte para gente: por que falar Espanhol seria uma conquista para você? ",
    "Você já PERDEU uma grande oportunidade pessoal/profissional por não saber falar Espanhol? Caso, sim, conte pra gente",
    'E o que te impede de resolver esse problema, e finalmente falar Espanhol?',
    'Você já está no grupo do whatsapp do evento? (caso não, confira seu e-mail)',
    'Você já participou de alguma semana de aulas gratuitas do VHE?'
], axis=1, errors='ignore')

df_pesquisa_captacao_l1_25 = df_pesquisa_captacao_l1_25.drop([
    "Nome",
    'Você trabalha?',
    "Qual foi a sua principal dúvida que você tinha antes de entrar no Portal VHE? Por onde começar e se deveria dar o primeiro passo pra fazer o curso.",
    "Seu whatsapp com DDD (opcional)",
    'Use a imaginação: descreva em três linhas como você imagina sua vida após realizar seu sonho através do Espanhol?',
    'Se você pudesse me pedir qualquer assunto para falar durante o evento, qual seria? '
], axis=1, errors='ignore')

In [3]:
df_pesquisa_captacao_l3_24.rename(columns={
    'Carimbo de data/hora': 'data_inscricao',
    'Nome de usuário': 'email',
    'Gênero': 'genero',
    'Faixa etária': 'faixa_etaria',
    'Qual o seu nível de escolaridade?': 'escolaridade',
    'Você trabalha com o que? ': 'profissao',
    'Qual a sua renda média? ': 'renda_media',
    'Por qual rede social você mais acompanha o VHE e a Profe Juliana?': 'onde_acompanha_conteudo',
    'Há quanto (mais ou menos) você acompanha as nossas redes sociais?': 'tempo_antes_redes_sociais',
    'Você fala outro idioma?': 'fala_outro_idioma',
    'E qual é o SEU nível no idioma?': 'nivel_idioma',
    'Porque você deseja chegar à fluência no Espanhol?': 'motivo_fluencia_espanhol',
    'Qual o maior problema que te impede de realizar esse sonho?': 'problema_aprender',
    'Você faz ou já fez algum curso de Espanhol?': 'fez_curso_espanhol',
    'Você já ouviu falar do meu curso completo, o "Portal VHE"? ': 'ouviu_falar_portal_vhe',
    'Caso você opte por fazer o meu curso ao final do evento, o quanto estaria disposto a investir para chegar à fluência no Espanhol em 6 meses com a minha ajuda?': 'investir_portal_vhe'  
}, inplace=True)

In [4]:
df_pesquisa_captacao_l4_24.rename(columns={
    'Carimbo de data/hora': 'data_inscricao',
    'Gênero': 'genero',
    'Faixa etária': 'faixa_etaria',
    'Qual o seu nível de escolaridade?': 'escolaridade',
    'Qual tipo de profissão você exerce?': 'profissao',
    'Qual a sua renda média? ': 'renda_media',
    'Por qual rede social você mais acompanha a Profe Juliana?': 'onde_acompanha_conteudo',
    'Há quanto (mais ou menos) você acompanha as nossas redes sociais?': 'tempo_antes_redes_sociais',
    'Você fala outro idioma?': 'fala_outro_idioma',
    'E qual é o SEU nível no Espanhol?': 'nivel_idioma',
    'Por que você deseja chegar à fluência no Espanhol?': 'motivo_fluencia_espanhol',
    'Depois de todo o conteúdo do evento, eu vou te convidar a fazer parte do meu curso completo, o Portal VHE. \n\nO quanto você estaria disposto a investir nesse curso que pode te levar a fluência em 6 meses?': 'investir_portal_vhe'
}, inplace=True)

In [5]:
df_pesquisa_captacao_l1_25.rename(columns={
    'Carimbo de data/hora': 'data_inscricao',
    'Nome de usuário': 'email',
    'Gênero': 'genero',
    'Faixa etária': 'faixa_etaria',
    'Qual o seu nível de escolaridade?': 'escolaridade',
    'Você trabalha com o que? ': 'profissao',
    'Qual a sua renda média (por mês)?': 'renda_media',
    'Por qual rede social você mais acompanha o VHE e a Profe Juliana?': 'onde_acompanha_conteudo',
    'Há quanto (mais ou menos) você acompanha as nossas redes sociais?': 'tempo_antes_redes_sociais',
    'Você faz ou já fez algum curso PAGO de Espanhol?': 'fez_curso_espanhol',
    'Você já fala outro idioma?': 'fala_outro_idioma',
    'E qual é o SEU nível no idioma?': 'nivel_idioma',
    'Por que você deseja chegar à fluência no Espanhol?': 'motivo_fluencia_espanhol',
    'Qual o problema que te impede de falar Espanhol hoje? ': 'problema_aprender',
    'Você já conhece o meu curso completo, "Portal VHE"? ': 'ouviu_falar_portal_vhe',
    'Quanto você estaria disposto(a) a investir em curso online que te levasse a fluência no Espanhol em 6 meses?': 'investir_portal_vhe'
}, inplace=True)

In [6]:
df_total = [
    df_pesquisa_captacao_l3_24,
    df_pesquisa_captacao_l4_24,
    df_pesquisa_captacao_l1_25
]

nomes = [
    "df_pesquisa_captacao_l3_24",
    "df_pesquisa_captacao_l4_24",
    "df_pesquisa_captacao_l1_25"
]

for nome, df in zip(nomes, df_total):
    print(f"\n=== {nome} ===")
    df.info()


=== df_pesquisa_captacao_l3_24 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1819 entries, 0 to 1818
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   data_inscricao             1819 non-null   object
 1   email                      1819 non-null   object
 2   genero                     1819 non-null   object
 3   faixa_etaria               1819 non-null   object
 4   escolaridade               1819 non-null   object
 5   profissao                  1819 non-null   object
 6   renda_media                1819 non-null   object
 7   onde_acompanha_conteudo    1819 non-null   object
 8   tempo_antes_redes_sociais  1819 non-null   object
 9   fala_outro_idioma          1819 non-null   object
 10  nivel_idioma               1819 non-null   object
 11  motivo_fluencia_espanhol   1819 non-null   object
 12  problema_aprender          1819 non-null   object
 13  fez_curso_espanhol         

In [7]:
for df in df_total:
    df['data_inscricao'] = pd.to_datetime(
        df['data_inscricao'].str.replace(' GMT-3', '', regex=False),
        format='%Y/%m/%d %I:%M:%S %p',
        errors='coerce'
    ).dt.normalize()

In [8]:
for nome, df in zip(nomes, df_total):
    print(f"\n=== {nome} ===")
    df.info()


=== df_pesquisa_captacao_l3_24 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1819 entries, 0 to 1818
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   data_inscricao             1819 non-null   datetime64[ns]
 1   email                      1819 non-null   object        
 2   genero                     1819 non-null   object        
 3   faixa_etaria               1819 non-null   object        
 4   escolaridade               1819 non-null   object        
 5   profissao                  1819 non-null   object        
 6   renda_media                1819 non-null   object        
 7   onde_acompanha_conteudo    1819 non-null   object        
 8   tempo_antes_redes_sociais  1819 non-null   object        
 9   fala_outro_idioma          1819 non-null   object        
 10  nivel_idioma               1819 non-null   object        
 11  motivo_fluencia_espanhol   1819 n

In [9]:
for nome, df in zip(nomes, df_total):
    print(f"\n=== {nome} ===")
    display(df.head())


=== df_pesquisa_captacao_l3_24 ===


,data_inscricao,email,genero,faixa_etaria,escolaridade,profissao,renda_media,onde_acompanha_conteudo,tempo_antes_redes_sociais,fala_outro_idioma,nivel_idioma,motivo_fluencia_espanhol,problema_aprender,fez_curso_espanhol,ouviu_falar_portal_vhe,investir_portal_vhe
0,2024-04-21,rachelsoares.ramos@gmail.com,Mulher,45 - 54,Superior completo,Funcionário público,"Entre R$5.000,01 e R$10.000,00",YouTube,Mais de 1 mês e menos de 3 meses,"Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Quero viajar e viver experiências marcantes me...,Não sei por onde começar a estudar,Não,Sim,"Posso investir até R$150,00 por mês"
1,2024-04-22,dariellymaelly71@gmail.com,Mulher,18 - 24,Médio completo,Funcionário público,"Até R$1.500,00",Instagram,Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Quero viajar e viver experiências marcantes me...,"Não tenho disciplina, não consigo estudar sem ...",Não,Não,Não tenho dinheiro para investir em curso
2,2024-04-22,paolabarretobs@gmail.com,Mulher,25 - 34,Superior completo,"Não trabalho, estou desempregrado(a)",Não tenho renda,Instagram,Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Intercâmbio,Tenho muita dificuldade com a pronúncia,Não,Não,Não tenho dinheiro para investir em curso
3,2024-04-22,victoria_san_oliveira@hotmail.com,Mulher,25 - 34,Pós-graduação completa,Autônomo/Empresário,"R$2.5001,00 à R$5.000,00",Instagram,Mais de 3 meses menos de 1 ano,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Quero viajar e viver experiências marcantes me...,Falta de um método eficiente,Sim,Não,"Posso investir até R$150,00 por mês"
4,2024-04-22,patyn.leite@gmail.com,Mulher,45 - 54,Superior completo,Autônomo/Empresário,"R$1.500,01 à R$2.500,00",YouTube,Menos de 2 semanas,"Sim, falo inglês",Iniciante - (Estou dando os primeiros passos),Morar no exterior,Tenho muita dificuldade com a pronúncia,Sim,Não,"Posso investir entre R$150,00 - R$250,00 por mês"



=== df_pesquisa_captacao_l4_24 ===


,data_inscricao,genero,faixa_etaria,escolaridade,profissao,renda_media,onde_acompanha_conteudo,tempo_antes_redes_sociais,fala_outro_idioma,nivel_idioma,motivo_fluencia_espanhol,investir_portal_vhe
0,2024-07-01,Mulher,45 - 54,Pós-graduação completa,CLT,"Entre R$5.000,01 e R$10.000,00",Instagram,Menos de 2 semanas,"Sim, falo inglês","Intermediário - (Já entendo algumas coisas, ma...",Quero viajar e viver experiências marcantes me...,Não tenho dinheiro para investir em curso
1,2024-07-01,Mulher,45 - 54,Pós-graduação completa,CLT,"Entre R$5.000,01 e R$10.000,00",Instagram,Menos de 2 semanas,"Sim, falo inglês","Intermediário - (Já entendo algumas coisas, ma...",Quero viajar e viver experiências marcantes me...,Não tenho dinheiro para investir em curso
2,2024-07-01,Mulher,25 - 34,Superior completo,"Não trabalho, estou desempregrado(a)","R$1.500,01 à R$2.500,00",Instagram,Mais de 1 mês e menos de 3 meses,"Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Conquistar um emprego melhor e ganhar mais,Não tenho dinheiro para investir em curso
3,2024-07-01,Mulher,55 - 64,Pós-graduação completa,Autônomo/Empresário,"Até R$1.500,00",Instagram,Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Conquistar um emprego melhor e ganhar mais,Não tenho dinheiro para investir em curso
4,2024-07-01,Mulher,55 - 64,Superior completo,Autônomo/Empresário,"R$1.500,01 à R$2.500,00",Não acompanho com frequência,Menos de 2 semanas,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Morar no exterior,"Posso investir até R$150,00 por mês"



=== df_pesquisa_captacao_l1_25 ===


,data_inscricao,email,genero,faixa_etaria,escolaridade,profissao,renda_media,onde_acompanha_conteudo,tempo_antes_redes_sociais,fez_curso_espanhol,fala_outro_idioma,nivel_idioma,motivo_fluencia_espanhol,problema_aprender,ouviu_falar_portal_vhe,investir_portal_vhe
0,2024-12-30,anapaula029@gmail.com,Mulher,44 - 55,Cursando Superior,CLT,"Até R$1.500,00",Instagram,Mais de 1 mês e menos de 3 meses,Fiz mas não concluí.,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Para viagem e trabalho,Pouco tempo para estudar,Não,"Até R$200,00 por mês"
1,2024-12-30,elisangelalemos20@yahoo.com,Mulher,35 - 44,Médio completo,Autônomo/Empresário,"R$1.500,01 à R$2.500,00",YouTube,Mais de 1 mês e menos de 3 meses,"Não, nunca fiz.","Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Aprender uma nova língua,Fala,Não,"Até R$200,00 por mês"
2,2024-12-30,concinha.silva@gmail.com,Mulher,44 - 55,Pós-graduação completa,Funcionário público,"Entre R$5.000,01 e R$ 10.000,00",Instagram,Mais de 1 ano,Fiz mas não concluí.,"Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Porque é um idioma que gosto muito.,A minha dedicação em estudar.,Sim,"Até R$200,00 por mês"
3,2024-12-30,teachermanuellafernandez@gmail.com,Mulher,35 - 44,Médio completo,CLT,"R$1.500,01 à R$2.500,00",YouTube,Mais de 1 ano,"Não, nunca fiz.","Sim, falo inglês","Intermediário - (Já entendo algumas coisas, ma...",Para o trabalho,Financeiro,Não,"Até R$200,00 por mês"
4,2024-12-30,lucianeinacio@yahoo.com.br,Mulher,44 - 55,Pós-graduação completa,Funcionário público,"Entre R$5.000,01 e R$ 10.000,00",Instagram,Menos de 2 semanas,"Não, nunca fiz.","Sim, falo inglês",Iniciante - (Estou dando os primeiros passos),Viagens,Método eficaz,Não,"Entre R$200,00 e R$300,00 por mês"


In [10]:
# Adiciona a coluna de identificação dos lançamentos
df_pesquisa_captacao_l3_24['lancamentos'] = 'L3-24'
df_pesquisa_captacao_l4_24['lancamentos'] = 'L4-24'
df_pesquisa_captacao_l1_25['lancamentos'] = 'L1-25'

In [11]:
# Concatenar os dataframes
df_pesquisa_captacao_lancamentos = pd.concat([
    df_pesquisa_captacao_l3_24,
    df_pesquisa_captacao_l4_24,
    df_pesquisa_captacao_l1_25
], ignore_index=True).copy()

df_pesquisa_captacao_lancamentos

,data_inscricao,email,genero,faixa_etaria,escolaridade,profissao,renda_media,onde_acompanha_conteudo,tempo_antes_redes_sociais,fala_outro_idioma,nivel_idioma,motivo_fluencia_espanhol,problema_aprender,fez_curso_espanhol,ouviu_falar_portal_vhe,investir_portal_vhe,lancamentos
0,2024-04-21,rachelsoares.ramos@gmail.com,Mulher,45 - 54,Superior completo,Funcionário público,"Entre R$5.000,01 e R$10.000,00",YouTube,Mais de 1 mês e menos de 3 meses,"Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Quero viajar e viver experiências marcantes me...,Não sei por onde começar a estudar,Não,Sim,"Posso investir até R$150,00 por mês",L3-24
1,2024-04-22,dariellymaelly71@gmail.com,Mulher,18 - 24,Médio completo,Funcionário público,"Até R$1.500,00",Instagram,Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Quero viajar e viver experiências marcantes me...,"Não tenho disciplina, não consigo estudar sem ...",Não,Não,Não tenho dinheiro para investir em curso,L3-24
2,2024-04-22,paolabarretobs@gmail.com,Mulher,25 - 34,Superior completo,"Não trabalho, estou desempregrado(a)",Não tenho renda,Instagram,Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Intercâmbio,Tenho muita dificuldade com a pronúncia,Não,Não,Não tenho dinheiro para investir em curso,L3-24
3,2024-04-22,victoria_san_oliveira@hotmail.com,Mulher,25 - 34,Pós-graduação completa,Autônomo/Empresário,"R$2.5001,00 à R$5.000,00",Instagram,Mais de 3 meses menos de 1 ano,"Não, o Espanhol será meu primeiro idioma estra...","Intermediário - (Já entendo algumas coisas, ma...",Quero viajar e viver experiências marcantes me...,Falta de um método eficiente,Sim,Não,"Posso investir até R$150,00 por mês",L3-24
4,2024-04-22,patyn.leite@gmail.com,Mulher,45 - 54,Superior completo,Autônomo/Empresário,"R$1.500,01 à R$2.500,00",YouTube,Menos de 2 semanas,"Sim, falo inglês",Iniciante - (Estou dando os primeiros passos),Morar no exterior,Tenho muita dificuldade com a pronúncia,Sim,Não,"Posso investir entre R$150,00 - R$250,00 por mês",L3-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7223,2025-01-28,orir.andrade@gmail.com,Homem,+55,Médio completo,Não trabalho,"Até R$1.500,00","Não acompanho, acabei de conhecer por um patro...",Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...","Perdido - (Nem sei por onde começar, mas quero...",Para expandir meus horizontes profissionais e ...,"Nenhum, apenas não tive a oportunidade de faze...","Não, nunca fiz.",Não,Não tenho como investir,L1-25
7224,2025-01-28,sandracabral942@gmail.com,Mulher,+55,Médio completo,Não trabalho,Não tenho renda,"Não acompanho, acabei de conhecer por um patro...",Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Pra conversar,Custo,"Não, nunca fiz.",Não,Não tenho como investir,L1-25
7225,2025-02-01,luziaalmeida14lu@gmail.com,Mulher,35 - 44,Médio completo,CLT,"Até R$1.500,00",YouTube,Conheci hoje,"Não, o Espanhol será meu primeiro idioma estra...","Perdido - (Nem sei por onde começar, mas quero...",Trabalho,Tempo,"Não, nunca fiz.",Não,Não tenho como investir,L1-25
7226,2025-02-09,akarinna66@gmail.com,Mulher,+55,Superior completo,Autônomo/Empresário,"Até R$1.500,00",Instagram,Menos de 2 semanas,"Não, o Espanhol será meu primeiro idioma estra...",Iniciante - (Estou dando os primeiros passos),Sim,Foco,"Não, nunca fiz.",Não,Não tenho como investir,L1-25


In [12]:
df_pesquisa_captacao_lancamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7228 entries, 0 to 7227
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   data_inscricao             7228 non-null   datetime64[ns]
 1   email                      4907 non-null   object        
 2   genero                     7228 non-null   object        
 3   faixa_etaria               7228 non-null   object        
 4   escolaridade               7228 non-null   object        
 5   profissao                  7228 non-null   object        
 6   renda_media                7228 non-null   object        
 7   onde_acompanha_conteudo    7228 non-null   object        
 8   tempo_antes_redes_sociais  7228 non-null   object        
 9   fala_outro_idioma          7228 non-null   object        
 10  nivel_idioma               7228 non-null   object        
 11  motivo_fluencia_espanhol   7228 non-null   object        
 12  proble

In [13]:
df_pesquisa_captacao_lancamentos["faixa_etaria"] = (
    df_pesquisa_captacao_lancamentos["faixa_etaria"]
    .replace({
        "45 - 54": "44 - 55",
        "55 - 64": "+55",
        "+65": "+55",
        "Conheci desde Dezembro de 2023.": "Desde Dez 2023"
    })
)

df_pesquisa_captacao_lancamentos["onde_acompanha_conteudo"] = (
    df_pesquisa_captacao_lancamentos["onde_acompanha_conteudo"]
    .replace({
        "Conheci AGORA, portanto, não acompanho com frequência": "Não acompanhava",
        "Não acompanho, acabei de conhecer por um patrocinado": "Não acompanhava",
        "Não acompanho com frequência": "Não acompanhava",
        "Não acompanho, um amigo me indicou": "Não acompanhava"
    })
)      

df_pesquisa_captacao_lancamentos["nivel_idioma"] = (
    df_pesquisa_captacao_lancamentos["nivel_idioma"]
    .replace({
        "Iniciante - (Estou dando os primeiros passos)": "Iniciante",
        "Intermediário - (Já entendo algumas coisas, mas ainda tenho dificuldades na fala)": "Intermediário",
        "Perdido - (Nem sei por onde começar, mas quero muito aprender)": "Perdido",
        "Avançado - (Entendo, leio, escrevo e falo muito bem)": "Avançado",
    })
)

df_pesquisa_captacao_lancamentos["renda_media"] = (
    df_pesquisa_captacao_lancamentos["renda_media"]
    .replace({
        "Até R$1.500,00": "Até 1.500",
        "R$1.500,01 à R$2.500,00": "De 1.500 a 2.500",
        "R$2.5001,00 à R$5.000,00": "De 2.500 a 5.000",
        "Entre R$5.000,01 e R$10.000,00": "De 5.000 a 10.000",
        "Entre R$5.000,01 e R$ 10.000,00": "De 5.000 a 10.000",
        "Acima de R$ 10.000,00 por mês": "Mais de 10.000",
        "Mais de R$10.000,00": "Mais de 10.000"
    })
)

df_pesquisa_captacao_lancamentos["onde_acompanha_conteudo"] = (
    df_pesquisa_captacao_lancamentos["onde_acompanha_conteudo"]
    .replace({
        "Tik Tok": "Outros",
        "Pesquisa no google": "Outros",
        "Blog": "Outros"
    })
)

In [14]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Mapas distintos por coluna
mapeamentos = {
    'problema_aprender': mapeamento_problema_aprender,
    'profissao': mapeamento_profissoes,
    'fala_outro_idioma': mapeamento_outros_idiomas,
    'motivo_fluencia_espanhol': mapeamento_motivo_fluencia,
    'escolaridade': mapeamento_escolaridade
}

for coluna, mapeamento in mapeamentos.items():
    nome_coluna_categoria = f"{coluna}_categoria"

    # Expandir mapeamento para esta coluna
    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)

    # Categorização com o mapeamento certo
    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(
        df_pesquisa_captacao_lancamentos[coluna],
        mapeamento_expandido,
        mapeamento_embeddings,
        model,
        threshold=0.6,
        desc=coluna
    )

escolaridade (busca direta): 100%|██████████████████████████████████████████████| 7228/7228 [00:00<00:00, 44585.41it/s]


In [15]:
# Exibir strings completas
pd.set_option("display.max_colwidth", None)

# Colunas categorizadas e respectivas colunas originais
categorias = {
    "problema_aprender": "problema_aprender_categoria",
    "profissao": "profissao_categoria",
    "fala_outro_idioma": "fala_outro_idioma_categoria",
    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",
    "escolaridade": "escolaridade_categoria"
}

# Categorias problemáticas
categorias_problema = ["Outros"]

# Loop para filtrar e exibir como DataFrame formatado
for original_col, categoria_col in categorias.items():
    for problema in categorias_problema:
        print(f"\n=== Valores de '{original_col}' categorizados como '{problema}' ===")

        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema
        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()

        # Envolver cada valor com aspas
        valores.index = [f'"{val}"' for val in valores.index]

        # Converter para DataFrame para visualização completa
        valores_df = valores.reset_index().head(300)
        valores_df.columns = ['valor_original', 'frequencia']

        # Exibir o resultado completo
        print(valores_df.to_string(index=False))


=== Valores de 'problema_aprender' categorizados como 'Outros' ===
                                                                                                   valor_original  frequencia
                                                                                                              "."          12
                                                                                                             ". "           3
                                                                                                              " "           1
                                                              "O problema é ainda em algumas conjugações verbais"           1
                                                                                                        "Atitude"           1
                                                                                      "Aprende sozinha é uitocil"           1
                                                  

In [16]:
df_pesquisa_captacao_lancamentos["email"] = df_pesquisa_captacao_lancamentos["email"].str.lower()

# Padronização de gênero na base de leads (captação)
map_genero = {
    "Homem": "Masculino",
    "Mulher": "Feminino"
}

# Aplicar transformação
df_pesquisa_captacao_lancamentos["genero"] = (
    df_pesquisa_captacao_lancamentos["genero"]
    .map(map_genero)
    .fillna(df_pesquisa_captacao_lancamentos["genero"])
)

In [17]:
df_pesquisa_captacao_lancamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7228 entries, 0 to 7227
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   data_inscricao                      7228 non-null   datetime64[ns]
 1   email                               4907 non-null   object        
 2   genero                              7228 non-null   object        
 3   faixa_etaria                        7228 non-null   object        
 4   escolaridade                        7228 non-null   object        
 5   profissao                           7228 non-null   object        
 6   renda_media                         7228 non-null   object        
 7   onde_acompanha_conteudo             7228 non-null   object        
 8   tempo_antes_redes_sociais           7228 non-null   object        
 9   fala_outro_idioma                   7228 non-null   object        
 10  nivel_idioma            

In [18]:
df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([
    'profissao',
    'fala_outro_idioma',
    'motivo_fluencia_espanhol',
    'escolaridade',
    'problema_aprender'
], axis=1, errors='ignore')

In [19]:
df_pesquisa_captacao_lancamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7228 entries, 0 to 7227
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   data_inscricao                      7228 non-null   datetime64[ns]
 1   email                               4907 non-null   object        
 2   genero                              7228 non-null   object        
 3   faixa_etaria                        7228 non-null   object        
 4   renda_media                         7228 non-null   object        
 5   onde_acompanha_conteudo             7228 non-null   object        
 6   tempo_antes_redes_sociais           7228 non-null   object        
 7   nivel_idioma                        7228 non-null   object        
 8   fez_curso_espanhol                  4907 non-null   object        
 9   ouviu_falar_portal_vhe              4907 non-null   object        
 10  investir_portal_vhe     

In [20]:
# Caminho de destino
caminho_csv = r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\dados\pesquisa_captacao_lancamentos.csv"

# Exportar para CSV com separador padrão (vírgula) e sem índice
df_pesquisa_captacao_lancamentos.to_csv(caminho_csv, index=False, encoding='utf-8-sig')

In [21]:
df_pesquisa_captacao_lancamentos.shape

(7228, 17)

In [22]:
df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop_duplicates(subset='email', keep='first')

df_pesquisa_captacao_lancamentos.shape

(4568, 17)

In [23]:
# Caminho de destino
caminho_csv = r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\dados\pesquisa_captacao_com_email.csv"

# Exportar para CSV com separador padrão (vírgula) e sem índice
df_pesquisa_captacao_lancamentos.to_csv(caminho_csv, index=False, encoding='utf-8-sig')

In [24]:
colunas_excluir = ["data_inscricao", "email"]

for coluna in df_pesquisa_captacao_lancamentos.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_captacao_lancamentos[coluna].value_counts(dropna=False))


Coluna: genero
genero
Feminino                 3390
Masculino                1154
Prefiro não responder      24
Name: count, dtype: int64

Coluna: faixa_etaria
faixa_etaria
44 - 55    1150
25 - 34    1051
35 - 44    1003
+55         697
14 - 17     345
18 - 24     322
Name: count, dtype: int64

Coluna: renda_media
renda_media
Não tenho renda      1150
De 1.500 a 2.500      993
Até 1.500             952
De 2.500 a 5.000      900
De 5.000 a 10.000     443
Mais de 10.000        130
Name: count, dtype: int64

Coluna: onde_acompanha_conteudo
onde_acompanha_conteudo
Instagram          2309
Não acompanhava    1172
YouTube            1038
Outros               49
Name: count, dtype: int64

Coluna: tempo_antes_redes_sociais
tempo_antes_redes_sociais
Conheci hoje                        1746
Menos de 2 semanas                  1198
Mais de 3 meses menos de 1 ano       610
Mais de 1 mês e menos de 3 meses     597
Mais de 1 ano                        417
Name: count, dtype: int64

Coluna: nivel_idi